In [ ]:
import numpy as np
import cv2
from google.colab.patches import cv2_imshow
cap = cv2.VideoCapture('slow_traffic_small.mp4')

feature_params = dict( maxCorners = 100,
                       qualityLevel = 0.3,
                       minDistance = 7,
                       blockSize = 7 )

lk_params = dict( winSize  = (15,15),
                  maxLevel = 2,
                  criteria = (cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 0.03))

color = np.random.randint(0,255,(100,3))
ret, old_frame = cap.read()
old_gray = cv2.cvtColor(old_frame, cv2.COLOR_BGR2GRAY)
p0 = cv2.goodFeaturesToTrack(old_gray, mask = None, **feature_params)
mask = np.zeros_like(old_frame)
while(1):
    ret,frame = cap.read()
    frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    p1, st, err = cv2.calcOpticalFlowPyrLK(old_gray, frame_gray, p0, None, **lk_params)
    good_new = p1[st==1]
    good_old = p0[st==1]
    for i,(new,old) in enumerate(zip(good_new,good_old)):
        a,b = new.ravel()
        c,d = old.ravel()
        mask = cv2.line(mask, (a,b),(c,d), color[i].tolist(), 2)
        frame = cv2.circle(frame,(a,b),5,color[i].tolist(),-1)
    img = cv2.add(frame,mask)

    cv2_imshow(img)
    k = cv2.waitKey(30) & 0xff
    if k == 27:
        break
    old_gray = frame_gray.copy()
    p0 = good_new.reshape(-1,1,2)

cv2.destroyAllWindows()
cap.release()

In [ ]:
import numpy as np
import cv2 as cv
cap = cv.VideoCapture(cv.samples.findFile("slow_traffic_small.mp4"))
ret, frame1 = cap.read()
prvs = cv.cvtColor(frame1, cv.COLOR_BGR2GRAY)
hsv = np.zeros_like(frame1)
hsv[..., 1] = 255
while(1):
    ret, frame2 = cap.read()
    if not ret:
        print('No frames available')
        break
    next = cv.cvtColor(frame2, cv.COLOR_BGR2GRAY)
    flow = cv.calcOpticalFlowFarneback(prvs, next, None, 0.5, 3, 15, 3, 5, 1.2, 0)
    mag, ang = cv.cartToPolar(flow[..., 0], flow[..., 1])
    hsv[..., 0] = ang*180/np.pi/2
    hsv[..., 2] = cv.normalize(mag, None, 0, 255, cv.NORM_MINMAX)
    bgr = cv.cvtColor(hsv, cv.COLOR_HSV2BGR)
    cv2_imshow(bgr)
    k = cv.waitKey(30) & 0xff
    if k == 27:
        break
    elif k == ord('s'):
        cv.imwrite('opticalfb.png', frame2)
        cv.imwrite('opticalhsv.png', bgr)
    prvs = next
cv.destroyAllWindows()

In [ ]:
import os
import cv2 as cv
A=dict();
A['label']=[]
A['files']=[]
D = '/content/drive/MyDrive/Cohn-Kanade/happy videos'
for i in os.listdir(D):
  A['label'].append(i)

# for i in os.listdir(D):
#   D1 = D + '/' + i 
#   img = cv.imread(D1, cv.IMREAD_COLOR)
#   A['label'].append(i)
#   A['files'].append(img)

In [ ]:
import os
from skimage import io
import numpy as np
from skimage.color import rgb2gray
D = '/content/drive/MyDrive/Cohn-Kanade/happy videos'
pics = os.listdir(D)
names = np.unique([x for x in pics])
l = {}
for name in names:
    l[name] = []
for file in os.listdir(D) :
  D1 = D + '/' + file
  D2 = os.listdir(D1)
  for j in D2:
    D3 = D1 + '/' + j
    img = cv.imread(D3, cv.IMREAD_COLOR)
    img=img[250:700,300:900]
    l[file].append(img)

In [ ]:
from google.colab.patches import cv2_imshow
l['S010'][1].shape

(450, 600, 3)

In [ ]:
# import cv2
# import numpy as np
# k={};
# for file in os.listdir(D) :
#   D1 = D + '/' + file
#   D2 = os.listdir(D1)
#   for j in D2:
#     D3 = D1 + '/' + j
#     k[D2].append(j)
# k

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import cv2

In [ ]:
happyvids = os.listdir('/content/drive/MyDrive/Cohn-Kanade/happy videos')
happy_features = np.zeros((len(happyvids) , 16))
y_happy = np.ones((len(happyvids)))
j = 0
for happ in happyvids :
    frames = os.listdir('/content/drive/MyDrive/Cohn-Kanade/happy videos/' + happ)
    i = 0 
    vidhist = np.zeros((len(frames)-1, 8))
    for frame in frames :
        if i == 0: 
            p1 = cv2.imread('/content/drive/MyDrive/Cohn-Kanade/happy videos/' + happ + "/" + frame )
            p1 = cv2.cvtColor(p1,cv2.COLOR_BGR2GRAY)
            p1 = p1[90:700,300:900]
            i += 1
            continue
        else:
            p2 = cv2.imread('/content/drive/MyDrive/Cohn-Kanade/happy videos/' + happ + "/" + frame )
            p2 = cv2.cvtColor(p2,cv2.COLOR_BGR2GRAY)
            p2 = p2[90:700,300:900]

        flow = cv2.calcOpticalFlowFarneback(p1,p2, None, 0.5, 3, 15, 3, 5, 1.2, 0)
        angle = flow[... ,1]
        vidhist[i-1] = np.histogram(angle.flatten(), bins=8)[0]        
        p1 = p2
        i += 1
    vidhist_mean = np.mean(vidhist, axis=0)
    vidhist_max = np.max(vidhist, axis=0)
    video_features = np.concatenate((vidhist_mean, vidhist_max), axis = 0)
    happy_features[j] = video_features
    j += 1

In [ ]:
surprisevids = os.listdir('/content/drive/MyDrive/Cohn-Kanade/surprise videos')
surprise_features = np.zeros((len(surprisevids) , 16))
y_surprise = np.zeros((len(surprisevids)))
j = 0
for surp in surprisevids :
    frames = os.listdir('/content/drive/MyDrive/Cohn-Kanade/surprise videos/' + surp)
    i = 0 
    vidhist = np.zeros((len(frames)-1, 8))
    for frame in frames :
        if i == 0: 
            p1 = cv2.imread('/content/drive/MyDrive/Cohn-Kanade/surprise videos/' + surp + "/" + frame )
            p1 = cv2.cvtColor(p1,cv2.COLOR_BGR2GRAY)
            p1 = p1[90:700,300:900]
            i += 1
            continue
        else:
            p2 = cv2.imread('/content/drive/MyDrive/Cohn-Kanade/surprise videos/' + surp + "/" + frame )
            p2 = cv2.cvtColor(p2,cv2.COLOR_BGR2GRAY)
            p2 = p2[90:700,300:900]

        flow = cv2.calcOpticalFlowFarneback(p1,p2, None, 0.5, 3, 15, 3, 5, 1.2, 0)
        angle = flow[... ,1]
        vidhist[i-1] = np.histogram(angle.flatten(), bins=8)[0]        
    
        p1 = p2
        i += 1
    vidhist_mean = np.mean(vidhist, axis=0)
    vidhist_max = np.max(vidhist, axis=0)
    video_features = np.concatenate((vidhist_mean, vidhist_max), axis = 0)
    surprise_features[j] = video_features
    j += 1

In [ ]:
import cv2 
import numpy as np
import matplotlib.pyplot as plt
import os
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC,LinearSVC
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import GridSearchCV

In [ ]:
print(surprise_features.shape)
X = np.concatenate((happy_features, surprise_features), axis = 0 )
y = np.concatenate((y_happy, y_surprise))
print(X.shape)
print(y.shape)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=2)

(30, 16)
(30, 16)
(60, 16)
(60,)


In [ ]:
y_train

array([1., 0., 1., 0., 0., 0., 1., 1., 0., 1., 1., 1., 1., 0., 1., 0., 1.,
       1., 0., 0., 1., 0., 1., 0., 0., 1., 1., 0., 1., 0., 0., 0., 1., 0.,
       1., 1., 0., 1., 1., 0., 1., 0.])

In [ ]:
parameters = {'gamma':[x / 1000000 for x in [1,2,3,4,5,6]], 'C':[0.01 , 0.1 , 1 , 10 , 100]}
svc1 = SVC()
clf1 = GridSearchCV(svc1, parameters)
clf1.fit(X_train, y_train)
clf1.best_params_

{'C': 0.01, 'gamma': 1e-06}

In [ ]:
clf = SVC(kernel = 'linear', C = 0.01, gamma = 0.000001)
clf.fit(X_train, y_train)

SVC(C=0.01, gamma=1e-06, kernel='linear')

In [ ]:
from sklearn.metrics import classification_report

In [ ]:
y_pred = clf.predict(X_test)
matrix = confusion_matrix(y_test,y_pred, labels=[1,0])
print('Confusion matrix : \n',matrix)

tp, fn = confusion_matrix(y_test,y_pred,labels=[1,0])
print('Outcome values : \n', tp, fn)

matrix = classification_report(y_test,y_pred,labels=[1,0])
print('Classification report : \n',matrix)

Confusion matrix : 
 [[7 1]
 [6 4]]
Outcome values : 
 [7 1] [6 4]
Classification report : 
               precision    recall  f1-score   support

           1       0.54      0.88      0.67         8
           0       0.80      0.40      0.53        10

    accuracy                           0.61        18
   macro avg       0.67      0.64      0.60        18
weighted avg       0.68      0.61      0.59        18



In [ ]:
clf.score(X_train, y_train)

1.0

In [ ]:
clf.score(X_test, y_test)

0.6111111111111112